# Data Cleaning and Transformation

In [1]:
import pandas as pd
import numpy as np

In [2]:
from sqlalchemy import create_engine

#### First, I'll need to import my data from PostgreSQL. To do that I'll first need to identify some parameters.

In [3]:
host = r'127.0.0.1'
db = r'MSDS610'
user = r'postgres'
pw = r'MacMVP2025$' 
port = r'5432'

In [4]:
db_conn = create_engine("postgresql://{}:{}@{}:{}/{}".format(user, pw, host, port, db))

#### Now, we can use the built-in pandas function for reading tables from SQL.

In [5]:
table_name = r'real_estate_data'
schema = r'raw'

real_estate2 = pd.read_sql_table(table_name, db_conn, schema)

In [6]:
real_estate2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1097629 entries, 0 to 1097628
Data columns (total 14 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   Serial Number     1097629 non-null  int64  
 1   List Year         1097629 non-null  int64  
 2   Date Recorded     1097627 non-null  object 
 3   Town              1097629 non-null  object 
 4   Address           1097578 non-null  object 
 5   Assessed Value    1097629 non-null  float64
 6   Sale Amount       1097629 non-null  float64
 7   Sales Ratio       1097629 non-null  float64
 8   Property Type     715183 non-null   object 
 9   Residential Type  699240 non-null   object 
 10  Non Use Code      313451 non-null   object 
 11  Assessor Remarks  171228 non-null   object 
 12  OPM remarks       13031 non-null    object 
 13  Location          298111 non-null   object 
dtypes: float64(3), int64(2), object(9)
memory usage: 117.2+ MB


In [7]:
real_estate2.head(10)

,Serial Number,List Year,Date Recorded,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type,Non Use Code,Assessor Remarks,OPM remarks,Location
0,220008,2022,01/30/2023,Andover,618 ROUTE 6,139020.0,232000.0,0.5992,Residential,Single Family,None,None,None,POINT (-72.343628962 41.728431984)
1,2020348,2020,09/13/2021,Ansonia,230 WAKELEE AVE,150500.0,325000.0,0.4630,Commercial,None,None,None,None,None
2,20002,2020,10/02/2020,Ashford,390 TURNPIKE RD,253000.0,430000.0,0.5883,Residential,Single Family,None,None,None,None
3,210317,2021,07/05/2022,Avon,53 COTSWOLD WAY,329730.0,805000.0,0.4096,Residential,Single Family,None,None,None,POINT (-72.846365959 41.781677018)
4,200212,2020,03/09/2021,Avon,5 CHESTNUT DRIVE,130400.0,179900.0,0.7248,Residential,Condo,None,None,None,None
5,200243,2020,04/13/2021,Avon,111 NORTHINGTON DRIVE,619290.0,890000.0,0.6958,Residential,Single Family,None,None,None,None
6,200377,2020,07/02/2021,Avon,70 FAR HILLS DRIVE,862330.0,1447500.0,0.5957,Residential,Single Family,None,None,None,None
7,22043,2022,03/15/2023,Beacon Falls,41 EDGEWOOD DRIVE,164170.0,285000.0,0.5760,Residential,Single Family,None,None,None,POINT (-73.053071989 41.439434021)
8,200109,2020,12/09/2020,Avon,57 FAR HILLS DRIVE,847520.0,1250000.0,0.6780,Residential,Single Family,None,None,None,None
9,2020180,2020,03/01/2021,Berlin,1539 FARMINGTON AVE,234200.0,130000.0,1.8015,Residential,Two Family,08 - Part Interest,None,None,None


#### As mentioned in my presentation, I'll need to make some changes to the data to prepare it for my ML model.

### Change 1 of 2 for week 3: Dropping unnesessary columns

In [8]:
real_estate3 = real_estate2.drop(columns= ['Date Recorded', 'Town', 'Address', 'Assessed Value', 'Sale Amount', 'Sales Ratio', 'Residential Type', 'Assessor Remarks', 'OPM remarks', 'Location'])
real_estate3.head(10)

,Serial Number,List Year,Property Type,Non Use Code
0,220008,2022,Residential,None
1,2020348,2020,Commercial,None
2,20002,2020,Residential,None
3,210317,2021,Residential,None
4,200212,2020,Residential,None
5,200243,2020,Residential,None
6,200377,2020,Residential,None
7,22043,2022,Residential,None
8,200109,2020,Residential,None
9,2020180,2020,Residential,08 - Part Interest


### Change 2 of 2 for week 3: Removing rows containing the list year of 2008

In [9]:
real_estate3 = real_estate3[~real_estate3['List Year'].isin(['2008'])]

In [10]:
real_estate3.head()

,Serial Number,List Year,Property Type,Non Use Code
0,220008,2022,Residential,None
1,2020348,2020,Commercial,None
2,20002,2020,Residential,None
3,210317,2021,Residential,None
4,200212,2020,Residential,None


#### Alright, now we've gotten started with our data transformation and will now upload an updated version of the table to SQL.

In [12]:
table_name = r'real_estate_clean'
schema = r'cleaned'

real_estate3.to_sql(table_name, con=db_conn, if_exists='replace', index=False, schema=schema, chunksize=1000, method='multi')

1097629